In [ ]:
!git clone https://github.com/phizaz/diffae

!pip install pytorch-lightning==1.5.10

!mkdir lmdb
!mkdir -p pytorch_fid/fid_score
!mkdir lpips

In [ ]:
import gdown

# https://drive.google.com/drive/folders/1-5zfxT6Gl-GjxM7z9ZO2AHlB70tfmF6V

file_id = '1-BBodaXHMrYFutdyHwokrE7vRof83fwH'

gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'ffhq256_autoenc.ckpt', quiet=False)


In [ ]:
import torch
from torchvision.transforms import functional as VF
from PIL import Image
from matplotlib import pyplot

In [ ]:
import os, sys

sys.path.append(os.path.abspath('./diffae/'))

# autoencoder

In [ ]:
from templates import ffhq256_autoenc, LitModel

In [ ]:
device = 'cuda'
conf = ffhq256_autoenc()
model = LitModel(conf)
state = torch.load(f'./ffhq256_autoenc.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)
model.ema_model.eval()
model.ema_model.to(device);

In [ ]:
img = Image.open('./diffae/imgs/sandy.JPG')
img = img.resize((256, 256)).convert('RGB')

x = VF.to_tensor(img)
x = x * 2. - 1.
x = x.unsqueeze(0).to(device)

with torch.no_grad():

    # https://github.com/phizaz/diffae/blob/master/experiment.py#L141
    # https://github.com/phizaz/diffae/blob/master/model/unet.py#L496

    cond = model.encode(x)

    # https://github.com/phizaz/diffae/blob/master/experiment.py#L147
    # https://github.com/phizaz/diffae/blob/master/diffusion/base.py#L670

    xT = model.encode_stochastic(x, cond, T=250)

    # https://github.com/phizaz/diffae/blob/master/experiment.py#L120
    # https://github.com/phizaz/diffae/blob/master/renderer.py#L43
    # https://github.com/phizaz/diffae/blob/master/diffusion/base.py#L181
    # https://github.com/phizaz/diffae/blob/master/diffusion/base.py#L533
    # https://github.com/phizaz/diffae/blob/master/diffusion/base.py#L483-L484

    pred = model.render(xT, cond, T=20)

cond.shape, xT.shape

In [ ]:
VF.to_pil_image(torch.cat([
    torch.clamp(x[0] * .5 + .5, 0., 1.),
    torch.clamp(xT[0] * .125 + .5, 0., 1.),
    torch.clamp(pred[0], 0., 1.)
], dim=2))